# LLM-Powered AI Agents for Building and Validating Computer Vision Pipelines

## Install required libraries

In [1]:
!pip install -q -U pynvml bitsandbytes transformers peft accelerate datasets scipy einops evaluate trl rouge_score

## Loading the required libraries

In [2]:
from datasets import load_dataset, Dataset
from transformers import (
    AutoModelForCausalLM,
    BitsAndBytesConfig,
    HfArgumentParser,
    AutoTokenizer,
    TrainingArguments,
    Trainer,
    GenerationConfig
)
from tqdm import tqdm
from trl import SFTTrainer
from huggingface_hub import interpreter_login
from pynvml import *
from functools import partial
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training, PeftModel
import torch
import time
import pandas as pd
import numpy as np
import transformers
import evaluate

In [3]:
import os
# disable Weights and Biases
os.environ['WANDB_DISABLED']="true"

## Loading dataset

In [4]:
df=pd.read_csv("./cva_gst_dataset/train.csv")

In [5]:
dataset = Dataset.from_pandas(df)

In [6]:
dataset[0]

{'prompt': '{"source_type": "rtsp", "source_location": "rtsp://192.168.1.124:8069/preview", "video_scale": {"width": 1597, "height": 518}, "caps_filter": "GBR", "detect": {"model": "/usr/share/models/onnx/detection/1.0/FP32/model_8007.xml", "device": "HDDL"}, "track": {"tracking_type": 0}, "classify": {"model": "/opt/models/intel/segmentation/2.0/FP32/model_3695.xml", "device": "GPU", "inference_region": "full-frame", "model_proc": "/home/user/.local/intel/model_proc/detection/model_3956.json"}, "inference": [{"model": "/home/user/.local/models/intel/classification/5.0/FP32/model_5775.xml", "device": "CPU", "inference_region": "roi-list"}, {"model": "/usr/share/models/onnx/recognition/2.1/FP16/model_7317.xml", "device": "HDDL", "inference_region": "roi-list"}, {"model": "/opt/models/intel/detection/3.0/FP16/model_9971.xml", "device": "CPU", "inference_region": "roi-list"}, {"model": "/usr/share/models/intel/segmentation/5.0/INT8/model_6368.xml", "device": "CPU", "inference_region": "ro

## Train Validation Split

In [7]:
train_val = dataset.train_test_split(test_size=0.1, seed=42)

In [8]:
dataset = {
    "train": train_val['train'],
    "validation": train_val['test']
}

## Create Bitsandbytes configuration

In [9]:
compute_dtype = getattr(torch, "float16")
bnb_config = BitsAndBytesConfig(
        load_in_4bit=True,
        bnb_4bit_quant_type='nf4',
        bnb_4bit_compute_dtype=compute_dtype,
        bnb_4bit_use_double_quant=False,
    )

device_map = {"": 0}

## Loading the Pre-Trained model

In [10]:
model_name='microsoft/phi-2'
base_model = AutoModelForCausalLM.from_pretrained(model_name, 
                                                      device_map=device_map,
                                                      quantization_config=bnb_config,
                                                      trust_remote_code=True,
                                                      use_auth_token=True)

/home/studio-lab-user/.conda/envs/default/lib/python3.9/site-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

## Tokenization

In [11]:
tokenizer = AutoTokenizer.from_pretrained(model_name,trust_remote_code=True,padding_side="left",add_eos_token=True,add_bos_token=True,use_fast=False)
tokenizer.pad_token = tokenizer.eos_token

In [12]:
eval_tokenizer = AutoTokenizer.from_pretrained(model_name, add_bos_token=True, trust_remote_code=True, use_fast=False)
eval_tokenizer.pad_token = eval_tokenizer.eos_token

In [13]:
def generate_text_from_model(model,prompt):
    toks = eval_tokenizer(prompt, return_tensors="pt")
    res = model.generate(**toks.to("cuda"), max_new_tokens=1000, do_sample=True,num_return_sequences=1,temperature=0.1,num_beams=1,top_p=0.95,).to('cpu')
    return eval_tokenizer.batch_decode(res,skip_special_tokens=True)

## Test the Model with Zero Shot Inferencing

In [14]:
def get_formatted_prompt(prompt):
    return f"Task: Convert the JSON configuration below into a valid gst-launch-1.0 pipeline command.\nInput JSON:\n{prompt}\nOutput:\n"

In [15]:
%%time
seed = 42
set_seed(seed)

index = 10

sample_prompt = dataset['validation'][index]['prompt']
sample_pipeline = dataset['validation'][index]['pipeline']

formatted_prompt = get_formatted_prompt(sample_prompt)
res = generate_text_from_model(base_model,formatted_prompt)
output = res[0].split('Output:\n')[1]

dash_line = '-'.join('' for x in range(100))
print(dash_line)
print(f'INPUT PROMPT:\n{formatted_prompt}')
print(dash_line)
print(f'GROUND TRUTH:\n{sample_pipeline}\n')
print(dash_line)
print(f'MODEL GENERATION - ZERO SHOT:\n{output}')

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


---------------------------------------------------------------------------------------------------
INPUT PROMPT:
Task: Convert the JSON configuration below into a valid gst-launch-1.0 pipeline command. Ensure all elements and properties are properly formatted.
Input JSON:
{"source_type": "file", "source_location": "/opt/data/mall/shelf_7/sensor_5.mp4", "video_crop": {"top": 0, "left": 53, "right": 3, "bottom": 233}, "caps_filter": "NV12", "frame_rate": "20/1", "detect": {"model": "/var/lib/models/tensorflow/classification/2.0/INT8/model_9886.xml", "device": "HDDL", "model_proc": "/home/user/.local/intel/model_proc/recognition/model_1389.json"}, "track": {"tracking_type": 2}, "classify": {"model": "/usr/local/models/pytorch/detection/3.0/FP32/model_5894.xml", "device": "GPU", "inference_region": "roi-list"}, "emit_signals": false, "sync": false, "drop": false}
Output:

---------------------------------------------------------------------------------------------------
GROUND TRUTH:
gst-

## Pre-processing dataset

In [16]:
def create_prompt_formats(sample):
    """
    Format various fields of the sample ('instruction','output')
    Then concatenate them using two newline characters 
    :param sample: Sample dictionnary
    """
    INTRO_BLURB = "This tool generates GStreamer pipeline commands from JSON configurations. Follow the input format to create valid gst-launch-1.0 commands."
    INSTRUCTION_KEY = "### Instruct: Convert the JSON configuration below into a valid gst-launch-1.0 pipeline command. Ensure all elements and properties are properly formatted."
    RESPONSE_KEY = "### Output:"
    END_KEY = "### End"
    
    blurb = f"\n{INTRO_BLURB}"
    instruction = f"{INSTRUCTION_KEY}"
    input_context = f"{sample['prompt']}" if sample["prompt"] else None
    response = f"{RESPONSE_KEY}\n{sample['pipeline']}"
    end = f"{END_KEY}"
    
    parts = [part for part in [blurb, instruction, input_context, response, end] if part]

    sample["text"] = "\n\n".join(parts)

    return sample

In [17]:
def get_max_length(model):
    conf = model.config
    max_length = None
    for length_setting in ["n_positions", "max_position_embeddings", "seq_length"]:
        max_length = getattr(model.config, length_setting, None)
        if max_length:
            print(f"Found max lenth: {max_length}")
            break
    if not max_length:
        max_length = 1024
        print(f"Using default max length: {max_length}")
    return max_length


def preprocess_batch(batch, tokenizer, max_length):
    """
    Tokenizing a batch
    """
    return tokenizer(
        batch["text"],
        max_length=max_length,
        truncation=True,
    )

In [18]:
def preprocess_dataset(tokenizer: AutoTokenizer, max_length: int, seed, dataset):
    """Format & tokenize it so it is ready for training
    :param tokenizer (AutoTokenizer): Model Tokenizer
    :param max_length (int): Maximum number of tokens to emit from tokenizer
    """
    
    # Add prompt to each sample
    print("Preprocessing dataset...")
    dataset = dataset.map(create_prompt_formats) #, batched=True)
    
    _preprocessing_function = partial(preprocess_batch, max_length=max_length, tokenizer=tokenizer)
    
    dataset = dataset.map(
        _preprocessing_function,
        batched=True
    )

    # Filter out samples that have input_ids exceeding max_length
    dataset = dataset.filter(lambda sample: len(sample["input_ids"]) < max_length)
    
    # Shuffle dataset
    dataset = dataset.shuffle(seed=seed)

    return dataset

In [19]:
## Pre-process dataset
max_length = get_max_length(base_model)
print(max_length)

train_dataset = preprocess_dataset(tokenizer, max_length, seed, dataset['train'])
eval_dataset = preprocess_dataset(tokenizer, max_length, seed, dataset['validation'])

Found max lenth: 2048
2048
Preprocessing dataset...


Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Map:   0%|          | 0/900 [00:00<?, ? examples/s]

Filter:   0%|          | 0/900 [00:00<?, ? examples/s]

Preprocessing dataset...


Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Filter:   0%|          | 0/100 [00:00<?, ? examples/s]

In [20]:
print(f"Shapes of the datasets:")
print(f"Training: {train_dataset.shape}")
print(f"Validation: {eval_dataset.shape}")
print(train_dataset)

Shapes of the datasets:
Training: (900, 5)
Validation: (100, 5)
Dataset({
    features: ['prompt', 'pipeline', 'text', 'input_ids', 'attention_mask'],
    num_rows: 900
})


## Setup PEFT for Fine-Tuning

In [21]:
def print_number_of_trainable_model_parameters(model):
    trainable_model_params = 0
    all_model_params = 0
    for _, param in model.named_parameters():
        all_model_params += param.numel()
        if param.requires_grad:
            trainable_model_params += param.numel()
    return f"trainable model parameters: {trainable_model_params}\nall model parameters: {all_model_params}\npercentage of trainable model parameters: {100 * trainable_model_params / all_model_params:.2f}%"

print(print_number_of_trainable_model_parameters(base_model))

trainable model parameters: 262364160
all model parameters: 1521392640
percentage of trainable model parameters: 17.24%


In [22]:
config = LoraConfig(
    r=32, #Rank
    lora_alpha=32,
    target_modules=[
        'q_proj',
        'k_proj',
        'v_proj',
        'dense'
    ],
    bias="none",
    lora_dropout=0.05,  # Conventional
    task_type="CAUSAL_LM",
)

# 1 - Enabling gradient checkpointing to reduce memory usage during fine-tuning
base_model.gradient_checkpointing_enable()

# 2 - Using the prepare_model_for_kbit_training method from PEFT
base_model = prepare_model_for_kbit_training(base_model)

peft_model = get_peft_model(base_model, config)

In [23]:
print(print_number_of_trainable_model_parameters(peft_model))

trainable model parameters: 20971520
all model parameters: 1542364160
percentage of trainable model parameters: 1.36%


## Train PEFT Adapter

In [24]:
output_dir = './peft_GST_pipeline_training_QLora/final-checkpoint'

peft_training_args = TrainingArguments(
    output_dir = output_dir,
    warmup_steps=1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=4,
    max_steps=100,
    learning_rate=2e-4,
    optim="paged_adamw_8bit",
    logging_steps=5,
    logging_dir="./logs",
    save_strategy="steps",
    save_steps=5,
    eval_strategy="steps",
    eval_steps=5,
    do_eval=True,
    gradient_checkpointing=True,
    report_to="none",
    overwrite_output_dir = 'True',
    group_by_length=True,
)

peft_model.config.use_cache = False

peft_trainer = transformers.Trainer(
    model=peft_model,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    args=peft_training_args,
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False),
)

In [25]:
peft_training_args.device

device(type='cuda', index=0)

In [26]:
peft_trainer.train()

Step,Training Loss,Validation Loss
10,1.300400,1.321981
20,0.977200,0.784137
30,0.698000,0.477612
40,0.518600,0.372802
50,0.408500,0.333547


Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.
Trainer.tokenizer is now deprecated. You should use Trainer.processing_class instead.


KeyboardInterrupt: 

In [28]:
# Free memory
del base_model
del peft_trainer
torch.cuda.empty_cache()